# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,-13.77,66,100,12.74,RU,1703026928
1,1,bang saphan,11.2126,99.5117,24.09,79,100,1.77,TH,1703026928
2,2,adamstown,-25.0660,-130.1015,23.03,78,99,7.21,PN,1703026928
3,3,guerrero negro,27.9769,-114.0611,21.20,53,83,5.89,MX,1703026928
4,4,mount pearl,47.5166,-52.7813,12.69,89,75,9.26,CA,1703026929


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 650,
    size = "Humidity",
    color="City",
    alpha= 0.5,
    scale= 1
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
187,187,le vauclin,14.5452,-60.8388,26.03,69,0,3.09,MQ,1703026960
214,214,sao joao da barra,-21.6403,-41.0511,26.05,82,0,2.46,BR,1703026963
216,216,al ghayzah,16.2079,52.1760,22.80,67,0,2.41,YE,1703026963
251,251,saint-pierre,-21.3393,55.4781,25.32,78,0,2.06,RE,1703026954
311,311,mukalla,14.5425,49.1242,22.32,61,0,2.77,YE,1703026760
502,502,les avirons,-21.2418,55.3394,23.55,78,0,2.06,RE,1703027007


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country","Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(17) 

,City,Country,Lat,Lng,Humidity,Hotel Name
187,le vauclin,MQ,14.5452,-60.8388,69,
214,sao joao da barra,BR,-21.6403,-41.0511,82,
216,al ghayzah,YE,16.2079,52.1760,67,
251,saint-pierre,RE,-21.3393,55.4781,78,
311,mukalla,YE,14.5425,49.1242,61,
502,les avirons,RE,-21.2418,55.3394,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'accommodation.hotel',
    "limit":1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index,"Lng"]
    lat = hotel_df.loc[index,"Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    print(name_address.url)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=6cfda7adf7c94aea85628c0adb68e865&filter=circle%3A-60.8388%2C14.5452%2C10000&bias=proximity%3A-60.8388%2C14.5452
le vauclin - nearest hotel: Hôtel Macabou
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=6cfda7adf7c94aea85628c0adb68e865&filter=circle%3A-41.0511%2C-21.6403%2C10000&bias=proximity%3A-41.0511%2C-21.6403
sao joao da barra - nearest hotel: Pousada Mediterrâneo
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=6cfda7adf7c94aea85628c0adb68e865&filter=circle%3A52.176%2C16.2079%2C10000&bias=proximity%3A52.176%2C16.2079
al ghayzah - nearest hotel: فندق تاج العرب
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=6cfda7adf7c94aea85628c0adb68e865&filter=circle%3A55.4781%2C-21.3393%2C10000&bias=proximity%3A55.4781%2C-21.3393
saint-pierre - nearest hotel: Tropic Hotel
https://api.geoapify.com

,City,Country,Lat,Lng,Humidity,Hotel Name
187,le vauclin,MQ,14.5452,-60.8388,69,Hôtel Macabou
214,sao joao da barra,BR,-21.6403,-41.0511,82,Pousada Mediterrâneo
216,al ghayzah,YE,16.2079,52.1760,67,فندق تاج العرب
251,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
311,mukalla,YE,14.5425,49.1242,61,فندق الجابري
502,les avirons,RE,-21.2418,55.3394,78,Floralys


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 650,
    size = "Humidity",
    color="City",
    alpha=0.5,
    hover_cols=["Hotel Name", "Country"]
)
# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)